# Wine market study

The client, Domaine des Croix, is looking to define the price of its wine bottles for the US market. They have retrieved a set of 130k bottles of wine, with grape varieties, countries and regions of production, vintages (i.e. years of production), as well as notes ("points") and descriptions from oenologists (wine specialists), and the price of all these bottles on the American market.

**The objective will be to make a presentation of the market analysis and the price you recommend for the client's wines.** The client is not a data analyst, but would like to understand the process. You will have to explain how the prices were set, without getting too technical, in other words: make it easy to understand.

You will find below some frames to guide you in this analysis. First, do the common framework. Then, you can follow one of these frames (Machine Learning or Business Intelligence). **Don't try to do both frames! To choose is to give up. The client prefers qualitative work to exhaustive work.**
But you can also go in other directions to answer the client's problem. If you have good ideas to propose to the client, they are obviously welcome. You're the data analyst now. 




# Data sets
- Dataset of 130k wines: https://github.com/murpi/wilddata/raw/master/wine.zip
- Dataset of the 14 Domaine des Croix wines: https://github.com/murpi/wilddata/raw/master/domaine_des_croix.csv


# Expected deliverables
The client would like an 8 minute presentation followed by 7 minutes of questions. 
The presentation will contain at least these elements:
- Reminder of the context and the problem
- Exploratory analysis of the data
- Methodology, tools and languages used
- Presentation of the technical part and the code created for this analysis
- Answer to the business question: price proposal or price range to the client to be correctly positioned against the competition on the American market

+ Link to the Tableau Presentation: https://public.tableau.com/app/profile/carlos.filipe.gon.alves.mour.o/viz/PriceWineRecommendationforUSMarket/Story1#1

In [1]:
import pandas as pd
import numpy as np

link = "https://github.com/murpi/wilddata/raw/master/wine.zip"
link_croix = 'https://github.com/murpi/wilddata/raw/master/domaine_des_croix.csv'
df = pd.read_csv(link)

In [ ]:
df.head()

In [ ]:
df.info()

In [3]:
df.describe()

,points,price
count,129957.000000,120964.000000
mean,88.446655,35.358363
std,3.039481,41.018605
min,80.000000,4.000000
25%,86.000000,17.000000
50%,88.000000,25.000000
75%,91.000000,42.000000
max,100.000000,3300.000000


##Start tokenization process to get:
+ date from 'title';
+ WordCloud from 'description'

In [ ]:
# tokenize column 'title' to get the year number and create new column with year #
import nltk
nltk.download('popular') #download the popular package because the most common libraries to do the process of tokenization
from nltk.tokenize import word_tokenize

## function to tokenize: 
+ def func_clean(x):

In [ ]:
### def function to tokenize and to remove stopwords/punctuation from some columns ###

from string import punctuation #import punctuation to use in the function

def func_clean(x):
  stopwords = nltk.corpus.stopwords.words('english') #variable stopwords to remove useless words, to use in the function
  # punctuation = list(punctuation) #variable punctuation to remove punctuation, to use in function BUT in this def function 
                                    #it can be comment but need to be uncomment if use without def function  
  x_words = nltk.word_tokenize(x.lower()) #tokenize to get words only and lower them all so it will be able to remove stopwords in Capital letter 
  cleaned_tokens = [token for token in x_words if token not in stopwords
                  and token not in punctuation]# new variable with function where it will be removed stopwords and punctuation (need to be IMPORT)
  return ' '.join(cleaned_tokens) # the function ' '.join() is to return a STR and not list

In [ ]:
# apply the def func_clean() to the column 'title'

df['title_clean'] = df['title'].apply(func_clean)

In [ ]:
#confirmation of the new column and tokenization worked 

df.head()

In [ ]:
# using regex method to get the year from the title_clean column

import re

df['year_list'] = df['title_clean'].str.findall(r'\b[2]+[0]+[0-9]+[0-9]|\b[1]+[9]+[0-9]+[0-9]')
df.head()

In [ ]:
### confirmation of some rows had 2 dates on it ###

df['year_list'].value_counts()

## function to get the max date: 
+ def last(x):

In [ ]:
#creation function to get the max value/date from the list because there some lists with 2 dates on it#

def last(x):
  for number in x:
    return max(x)

df['year_final'] = df['year_list'].apply(last)

In [ ]:
# confirmation the dates now are correct and % of dates

df['year_final'].value_counts(dropna = False,
                              normalize = True)*100

## DF to get the oldest/newest wine date

In [ ]:
## removing 0 from "year_final" column

df_old_new_wines = df

In [ ]:
#Transform year_final in int from str

df_old_new_wines['year_final'] = pd.to_numeric(df_old_new_wines['year_final']) #.convert_dtypes(convert_integer = True)
df_old_new_wines['year_final'] = df_old_new_wines['year_final'].replace(np.nan, 0, regex=True) #NaN need to be replaced by 0 in order to transform in numeric all column
df_old_new_wines['year_final'] = df_old_new_wines['year_final'].astype(int)
df_old_new_wines.info()

In [ ]:
df_old_new_wines = df_old_new_wines[df_old_new_wines['year_final'] != 0]

In [ ]:
# confirmation of % of years and 0 was removed

df_old_new_wines['year_final'].value_counts(dropna = False,
                              normalize = True)*100

In [ ]:
# get the oldest/newest wine date

print('Newest wine year = ',df_old_new_wines['year_final'].max())
print('Oldest wine year = ',df_old_new_wines['year_final'].min())

##Check the best/worst wine by price/score

In [ ]:
## to check the wine with best/worst score and prices
df.sort_values(by=['points', 'price'],
                       ascending = False)

In [ ]:
# remove title_clean and year_list from the df

df.drop(columns = ['title_clean', 'year_list'],
        inplace = True)

df.head()

In [ ]:
# get a value counts from country just to have an idea of the % of countries represent on the wines #
df['country'].value_counts(normalize = True, dropna = False)*100

## Correlation analyse

In [ ]:
# did pairplot and heatmap just to understand if there is correlation between Points(score) and Price

import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(df.corr(),
            annot = True)

In [ ]:
sns.pairplot(df)

#Filter code Template

In [ ]:
# just to check columns if needed

df.columns

In [ ]:
# this code is just to filtrate some extra information I need to see quick
df[df['price'] == 3300]

##Download US Wines to use on Tableau

In [ ]:
### download of table dates to use in Tableau ###

# from openpyxl import load_workbook

# df.to_excel(r'/content/drive/MyDrive/Checkpoint Exams/Wine_with_index.xlsx')#, index = False)

##Download Burgundy dataset

In [ ]:
# create of new dataset to use on WordCloud

df_burgundy = df[df['province'] == 'Burgundy']
df_burgundy

In [ ]:
### download of table dates to use in Tableau ###

# from openpyxl import load_workbook

# df_burgundy.to_excel(r'/content/drivport load_we/MyDrive/Checkpoint Exams/Wine_Burgundy_index.xlsx')#, index = False)

##Download Pinot variety Wines

In [ ]:
# check how many variety of wines exist

df['variety'].value_counts(dropna = False)

In [ ]:
# looking for others Pinot Noir in the df

df_pinot = df[df['variety'].str.contains('inot') == True]
df_pinot['variety'].value_counts()

In [ ]:
## confirm which variety is most common on province 'Burgundy'

df_pinot[(df_pinot['province'] == 'Burgundy')].groupby('variety').agg({'variety':'count'})

In [ ]:
### download of table dates to use in Tableau ###

# from openpyxl import load_workbook

# df_pinot.to_excel(r'/content/drive/MyDrive/Checkpoint Exams/Wine_Pinot.xlsx')#, index = False)

In [ ]:
# new dataset with only Pinot Noir variety to use on WordCloud and work in Tableau if needed

df_pinot_noir = df[(df['variety'] == 'Pinot Noir') | (df['variety'] == 'Chardonnay')]

df_pinot_noir.variety.value_counts()

In [ ]:
### download of table dates to use in Tableau ###

# from openpyxl import load_workbook

# df_pinot_noir.to_excel(r'/content/drive/MyDrive/Checkpoint Exams/Wine_Pinot_Noir_Only.xlsx')#, index = False)

In [ ]:
### first filter used on Tableau

# df_pinot_burgundy = df_pinot_noir[(df_pinot_noir['province'] == 'Burgundy') &
#                                   (df_pinot_noir['year_final'] >= 2014) &
#                                   (df_pinot_noir['year_final'] < 2020)]


### second filter used on Tableau and see if makes difference
df_pinot_burgundy = df_pinot_noir[(df_pinot_noir['province'] == 'Burgundy') &
                                  (df_pinot_noir['year_final'] >= 2014) &
                                  (df_pinot_noir['year_final'] < 2020) &
                                  (df_pinot_noir['points'] >= 90) &
                                  (df_pinot_noir['points'] <= 95)]

df_pinot_burgundy

In [ ]:
### download of table dates to use in Tableau ### TABLE WITH PINOT NOIR, BURGUNDY AND DATES BETWEEN 2014 and 2019

# from openpyxl import load_workbook

# df_pinot_burgundy.to_excel(r'/content/drive/MyDrive/Checkpoint Exams/Wine_Pinot_Burgundy_Final.xlsx')#, index = False)

# File Domaine_des_Croix

In [ ]:
import pandas as pd
import numpy as np

link = "https://github.com/murpi/wilddata/raw/master/wine.zip"
link_croix = 'https://github.com/murpi/wilddata/raw/master/domaine_des_croix.csv'
df_croix = pd.read_csv(link_croix)

In [ ]:
df_croix

In [ ]:
df_croix.info()

In [ ]:
df_croix.describe()

In [ ]:
# apply the def func_clean() to the column 'title'

df_croix['title_clean'] = df_croix['title'].apply(func_clean)

In [ ]:
# using regex method to get the year from the title_clean column

import re

df_croix['year_list'] = df_croix['title_clean'].str.findall(r'\b[2]+[0]+[0-9]+[0-9]|\b[1]+[9]+[0-9]+[0-9]')
df_croix

In [ ]:
#apply function last(x) to get the max value/date from the list because there some lists with 2 dates on it#

# def last(x):
#   for number in x:
#     return max(x)

df_croix['year_final'] = df_croix['year_list'].apply(last)

In [ ]:
# transform values from column year_final from str to int

df_croix['year_final'] = pd.to_numeric(df_croix['year_final']) #.convert_dtypes(convert_integer = True)
df_croix['year_final'] = df_croix['year_final'].replace(np.nan, 0, regex=True)
df_croix['year_final'] = df_croix['year_final'].astype(int)
df_croix.info()

In [ ]:
df_croix.describe()

In [ ]:
# get the max, min e total of wines per year

print(df_croix.groupby('year_final').agg({'points':'min'}))
print(df_croix.groupby('year_final').agg({'points':'max'}),'\n')
df_croix['year_final'].value_counts()

##Download Domain des Croix Dataset

In [ ]:
# from openpyxl import load_workbook

# df_croix.to_excel(r'/content/drive/MyDrive/Checkpoint Exams/Wine_Domain_Croix.xlsx')#, index = False)

# Word Cloud

###Creation of mask

In [ ]:
# import library to be able to upload mask with wine shape

from PIL import Image

wine_mask = np.array(Image.open("/content/drive/MyDrive/Checkpoint Exams/wine_mask_xgk1tq.png"))
wine_mask

In [ ]:
# because the image is black (RGB = 0) it needs to be converted in white (RGB = 255)
# 0 -> represents the "intensity" of the pixel

def transform_format(val):
    if val == 0:
        return 255
    else:
        return val

In [ ]:
# Transform your mask into a new one that will work with the function:
transformed_wine_mask = np.ndarray((wine_mask.shape[0],
                                    wine_mask.shape[1]),
                                   np.int32)

for i in range(len(wine_mask)):
    transformed_wine_mask[i] = list(map(transform_format, wine_mask[i]))

In [ ]:
# Confirmation of the transformation of your mask
transformed_wine_mask

##Word Cloud

In [ ]:
# import libraries

from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# apply func_clean(x) to tokenize the description

df['description_clean'] = df['description'].apply(func_clean)

In [ ]:
# create word cloud image from the description of all US market wine list 

text = df['description_clean'].values 

wordcloud = WordCloud(width = 600,
                      height = 300,
                      min_font_size= 6,
                      background_color="white",
                      mask = transformed_wine_mask, # wine mask
                      contour_width=3, 
                      contour_color='firebrick').generate(str(text)) #colour and text to be insert in the delimitation

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
# Save the image in a folder:

# wordcloud.to_file("/content/drive/MyDrive/Checkpoint Exams/Wine_WordCloud.png")

### dataset Pinot Noir (df_pinot_noir)

In [ ]:
df_pinot_noir['description_clean'] = df_pinot_noir['description'].apply(func_clean)

In [ ]:
# create word cloud image from Pinot Noir wines only from the US market list

text = df_pinot_noir['description_clean'].values 

wordcloud = WordCloud(width = 600,
                      height = 300,
                      min_font_size= 6,
                      background_color="white",
                      mask = transformed_wine_mask,
                      contour_width=3, 
                      contour_color='firebrick').generate(str(text))

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
# # Save the image in the img folder:

# wordcloud.to_file("/content/drive/MyDrive/Checkpoint Exams/Wine_Pinot_Noir_WordCloud.png")

### dataset Burgundy (df_burgundy)

In [ ]:
df_burgundy['description_clean'] = df_burgundy['description'].apply(func_clean)

In [ ]:
# create word cloud image from Burgundy province wines only from the US market list

text = df_burgundy['description_clean'].values 

wordcloud = WordCloud(width = 600,
                      height = 300,
                      min_font_size= 6,
                      background_color="white",
                      mask = transformed_wine_mask,
                      contour_width=3, 
                      contour_color='firebrick').generate(str(text))

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
# # Save the image in the img folder:

# wordcloud.to_file("/content/drive/MyDrive/Checkpoint Exams/Wine_Burgundy_WordCloud.png")

### dataset df_croix

In [ ]:
# apply func_clean(x) to tokenize the description

df_croix['description_clean'] = df_croix['description'].apply(func_clean)

In [ ]:
# create word cloud image from client's list

text = df_croix['description_clean'].values 

wordcloud = WordCloud(width = 600,
                      height = 300,
                      min_font_size= 6,
                      background_color="white",
                      mask = transformed_wine_mask,
                      contour_width=3, 
                      contour_color='firebrick').generate(str(text))

plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
# # Save the image in the img folder:

# wordcloud.to_file("/content/drive/MyDrive/Checkpoint Exams/Wine_Croix_WordCloud.png")